In [1]:
import pypyodbc

server = '192.168.0.21'
port = '4900'
database = 'ICAM'
username = 'admin'
password = 'Clip_SERENA'


cnxn = pypyodbc.connect('DRIVER={HFSQL}; \
                        Server Name =' + server + '; \
                        Server Port=' + port + '; \
                        DATABASE=' + database + '; \
                        UID=' + username + '; \
                        PWD=' + password)
                        #Trusted_Connection=yes;

# cnxn = pypyodbc.connect(r'DRIVER={HFSQL};ANA=C:\Program Files (x86)\Clip Industrie\CLIPPER 7\CLIPPER7.wd7\CLIPPER7.WDD;REP=C:\Clipindustrie\Public\Bases\BAC A SABLE')

crsr = cnxn.cursor()

table_name = 'CHARGE'
#cf_name = 'C6800'
semaine = '12'

#query = 'SELECT * FROM {}'.format(table_name)
#query = 'SELECT * FROM ' +table_name+ ' WHERE NAF < 1000'
query = "SELECT * FROM " +table_name+ " WHERE DAT like '20240621' AND COFRAIS like 'C35-2' LIMIT 2"
#COFRAIS like '" +cf_name+ "' AND

records = crsr.execute(query).fetchall()
print(records)

# tables = crsr.tables()
# Alltables = list(tables)
# print(Alltables)

#columns_list = list(crsr.columns())
#columns_list

[(516, datetime.date(2024, 6, 21), None, 'C35-2', 'ATCRM', 'C35-2                         ', 5.743507, 0.0, 'EWELL', 40211, 427838), (35668, datetime.date(2024, 6, 21), None, 'C35-2', 'ATCRM', 'C35-2                         ', 8.318182, 0.0, 'EWELL', 40212, 427850)]


#### one_row = crsr.fetchone()
type(one_row)